In [ ]:
import autogen

llm_config = {"config_list": [{
    "model": "gpt-3.5-turbo",
    "cache_seed": None
}]}

bob = autogen.AssistantAgent(
    name="Bob",
    system_message="""
        First, tell a joke. After Alice gives feedback, improve 
        the joke. After you tell the improved joke, say 'TERMINATE'.
    """,
    llm_config=llm_config,
)

alice = autogen.AssistantAgent(
    name="Alice",
    system_message="Criticise the joke.",
    llm_config=llm_config
)

def termination_message(msg):
    return "TERMINATE" in str(msg.get("content", ""))

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    code_execution_config={"use_docker": False},
    is_termination_msg=termination_message,
    human_input_mode="NEVER"
)

groupchat = autogen.GroupChat(
    agents=[bob, alice, user_proxy],
    messages=[],
    # speaker_selection_method="round_robin"
)

manager = autogen.GroupChatManager(
    groupchat=groupchat, 
    code_execution_config={"use_docker": False},
    llm_config=llm_config,
    is_termination_msg=termination_message
)

user_proxy.initiate_chat(
    manager,
    message="Tell a unique joke about cats"
)